# 数据预处理	
*	处理RNA

In [2]:
import numpy as np
import pandas as pd

In [26]:
RNA=pd.read_csv('/cluster/home/futing/ref_genome/hg38_gencode/genebed/gencode.v43.gene.bed',sep='\t',header=None)
RNA=RNA.loc[:,[0,1,2,3]]
RNA.columns=['chr','start','end','gene']
RNA['ENSG']=RNA['gene'].str.split('.').str[0]
RNA=RNA.loc[~RNA['ENSG'].duplicated(),:]
RNA=RNA.drop(columns=['gene'])

In [28]:
expr=pd.read_csv('/cluster2/home/futing/Project/panCancer/CRC/RNA/FPKM.protein_lncRNA.matrix_v2.log2.qq.rename.noERCC',sep='\t')
expr['ENSG']=expr['ENSGID'].str.split('.').str[0]
expr=expr.loc[~expr['ENSG'].duplicated(),:]#重复的都是PAR_Y结尾的
expr=expr.drop(columns=['ENSGID'])
expr['ENSG'].duplicated().sum()

0

In [29]:
expr.shape,RNA.shape

((27289, 81), (62656, 4))

In [37]:
pheno=pd.merge(RNA,expr,on='ENSG',how='inner')
pheno.shape

(26907, 84)

In [39]:
pheno.columns = [
    col[:-1] + '_Normal' if col.endswith('N') else
    col[:-1] if col.endswith('T') else
    col
    for col in pheno.columns
]

In [34]:
covariate=pd.read_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC/covariate/covariate.txt',sep='\t')
covariate.head()

,Unnamed: 0,09-376,11-1026,11-1074,11-1074_Normal,11-51,11-51_Normal,11-52,11-63,11-63_Normal,...,14-1083,14-1251,14-132,14-151,14-258,14-275,14-326,14-442,14-494,14-810
0,element,1.051792e+08,1.027580e+08,9.549728e+07,9.027582e+07,6.117425e+07,1.130951e+08,9.472309e+07,1.046700e+08,9.514518e+07,...,1.092722e+08,1.544300e+08,1.046870e+08,1.519218e+08,9.892819e+07,1.040042e+08,1.080451e+08,1.006462e+08,1.061229e+08,9.624084e+07
1,PC1,6.182690e-02,-1.610580e-02,7.679660e-02,1.147090e-01,1.359660e-01,1.689080e-01,1.200620e-01,-2.176520e-01,-2.373330e-02,...,-4.359850e-02,1.496430e-01,4.470440e-02,1.629930e-01,4.347490e-02,-2.044510e-01,-1.875640e-02,-2.939140e-01,7.236070e-02,1.215400e-01
2,PC2,-1.598650e-02,-8.974410e-03,-4.557630e-01,-4.621260e-01,5.266200e-01,5.389440e-01,-8.417480e-03,-7.287000e-03,-2.365810e-02,...,6.098930e-03,-8.804700e-03,-2.555110e-02,-3.355670e-02,-4.449420e-03,9.171540e-03,-7.026200e-03,1.989920e-02,3.934070e-03,-3.067150e-02
3,PC3,-6.026780e-02,-4.722770e-02,4.725060e-01,4.657210e-01,3.644130e-01,3.659040e-01,-1.041170e-01,-2.208930e-01,-2.952340e-01,...,-5.173430e-02,-9.256480e-02,-7.692250e-02,-1.229950e-01,-6.969270e-02,2.951690e-03,-4.292390e-02,4.029790e-02,-6.801830e-02,-1.104070e-01
4,PC4,-6.271210e-02,-5.920880e-02,1.452320e-01,1.491720e-01,1.385470e-01,1.411690e-01,-1.065940e-01,5.896080e-01,6.538930e-01,...,-7.041810e-02,-6.132770e-02,-7.748630e-02,-7.263740e-02,-7.321950e-02,-6.252760e-02,-8.051950e-02,-8.890650e-02,-8.346450e-02,-4.914510e-02


In [42]:
covariate.columns[1:]

Index(['09-376', '11-1026', '11-1074', '11-1074_Normal', '11-51',
       '11-51_Normal', '11-52', '11-63', '11-63_Normal', '11-927', '11-983',
       '12-251', '13-1317', '13-1320', '13-1321', '13-1334', '13-1410',
       '13-1413', '13-1483', '13-257', '13-542', '13-545', '14-1083',
       '14-1251', '14-132', '14-151', '14-258', '14-275', '14-326', '14-442',
       '14-494', '14-810'],
      dtype='object')

In [53]:
pheno1 = pheno.loc[:, ['chr', 'start', 'end', 'ENSG'] + covariate.columns[1:].tolist()]
pheno1=pheno1.rename(columns={'ENSG':'gene'})
pheno1.loc[:,'chr'] = pheno1.loc[:,'chr'].str.replace('chr', '', regex=False)

In [45]:
pheno1.shape

(26907, 36)

In [54]:
pheno1.to_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC/phenotype/expr.bed',sep='\t',index=False)